<a href="https://colab.research.google.com/github/dipnarayan501/Deep_learning/blob/main/Question_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.datasets import mnist, fashion_mnist
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
%pip install wandb -q
import wandb
wandb.login()

wandb: Currently logged in as: moni6264 (use `wandb login --relogin` to force relogin)


True

In [3]:
def prepare_dataset():
    (X, y), (X_test, y_test) = fashion_mnist.load_data()

    # Reshaping the data matrices
    X = X.reshape(X.shape[0], -1)
    X_test = X_test.reshape(X_test.shape[0], -1)

    # Normalizing the pixel intensities
    X = X/255.0
    X_test = X_test/255.0
    
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)
    
    y_train_encode = np.zeros((10,y_train.shape[0]))
    y_train_encode[y_train, np.array(list(range(y_train.shape[0])))] = 1

    y_val_encode = np.zeros((10,y_val.shape[0]))
    y_val_encode[y_val, np.array(list(range(y_val.shape[0])))] = 1

    
    
    print("Number of examples in training set")
    print(X_train.shape)
    print(y_train_encode.shape)
    print(y_train.shape)
    
    print("No of examples in validation set")
    print(X_val.shape)
    print(y_val.shape)
    print("No of examples in test set")
    print(X_test.shape)
    print(y_test.shape)
    
    return X_train.T, y_train_encode, y_train.T ,X_val.T, y_val.T, X_test.T, y_test.T, y_val_encode


X_train, y_train_encode, y_train ,X_val, y_val, X_test, y_test,y_val_encode = prepare_dataset() 

Number of examples in training set
(54000, 784)
(10, 54000)
(54000,)
No of examples in validation set
(6000, 784)
(6000,)
No of examples in test set
(10000, 784)
(10000,)


In [4]:
num_classes = 10
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
               
wandb.init(project="test_100", entity="moni6264", name="log_images")
indices = [list(y_train.T).index(i) for i in range(num_classes)]
images = []
captions = []
for index in indices:
    images.append(X_train.T[index].reshape((28, 28)))
    captions.append(class_names[y_train.T[index]])

# Log one sample image of each class to wandb
wandb.log({"Image from each class": [wandb.Image(image, caption=caption) for image, caption in zip(images, captions)]})

In [5]:
def sigmoid(x):
    return 1. / (1.+np.exp(-x))
def relu(x):
    return np.maximum(0,x)
def tanh(x):
    return np.tanh(x)
def softmax(x):
    soft = np.zeros(x.shape)
    for i in range(0, x.shape[1]):
        numr = np.exp(x[:, i])
        soft[:, i] = numr/np.sum(numr)
    return soft

#Derivatives
def sigmoid_derivative(x):
    return sigmoid(x) * (1-sigmoid(x))
def relu_derivative(x):
    return 1*(x>0) 
def tanh_derivative(x):
    return (1 - (np.tanh(x)**2))

def softmax(x):
    soft = np.zeros(x.shape)
    for i in range(0, x.shape[1]):
        numr = np.exp(x[:, i])
        soft[:, i] = numr/np.sum(numr)
    return soft

def initialize_parameters(input_layer, hidden_layer, output_layer,initilisation):
    weights = []
    biases  = []
    layers = [input_layer] + hidden_layer + [output_layer]
    for i in range(len(hidden_layer)+1):
        if initilisation == 'random':
            weights.append(np.random.rand(layers[i+1], layers[i])*0.01)
        if initilisation == 'xavier':
            weights.append(np.random.randn(layers[i+1],layers[i])*np.sqrt(2/layers[i+1]))
        biases.append(np.random.randn(layers[i+1],1))
    return weights, biases

In [7]:
import numpy as np
from sklearn.metrics import mean_squared_error

class ANN:

    def __init__(self,input_layer,hidden_layer,output_layer ,initialisation, activation,loss_function):
           
        self.input_layer   = input_layer
        self.output_layer  = output_layer
        self.hidden_layer   = hidden_layer
        self.activation = activation
        self.loss_function = loss_function
        self.weights    = []
        self.biases     = []
        self.initialisation = initialisation
        self.weights,self.biases = initialize_parameters(self.input_layer,self.hidden_layer,self.output_layer,self.initialisation)

    
    
    #Forward_propogation
    def forward_propagation(self,input):

        self.A = []
        self.H  = []
        k=0
        
        self.A.append(np.matmul(self.weights[k],input)+self.biases[k])
        if self.activation == 'sigmoid':
          self.H.append(sigmoid(self.A[k]))
        elif self.activation == 'tanh':
          self.H.append(tanh(self.A[k]))
        elif self.activation == 'relu': 
          self.H.append(relu(self.A[k])) 


        for k in range(1,len(self.hidden_layer)):
            self.A.append(np.matmul(self.weights[k],self.H[k-1])+self.biases[k])
            if self.activation == 'sigmoid':
              self.H.append(sigmoid(self.A[k]))
            elif self.activation == 'tanh':
              self.H.append(tanh(self.A[k]))
            elif self.activation == 'relu': 
              self.H.append(relu(self.A[k])) 
  


        k=len(self.hidden_layer)
        self.A.append(np.matmul(self.weights[k],self.H[k-1])+self.biases[k])
        self.H.append(softmax(self.A[k]))

        return self.H[-1]
  #Backward propogation
    def back_propagation(self,X_train,y_train):
        g_a  = [0]*(len(self.hidden_layer)+1)
        g_h  = [0]*(len(self.hidden_layer)+1)
        g_w  = [0]*(len(self.weights))
        g_b  = [0]*(len(self.biases))

        n_samples = X_train.shape[1]  # Change depending on the dimensions of data


        for k in reversed(range(len(self.hidden_layer)+1)):
            if k == len(self.hidden_layer):
              if self.loss_function == 'cross_entropy':
                  g_a[k] = self.H[k]  - y_train  # keep or remove T depending on the dimensions of data
              elif self.loss_function == 'square_loss': 
                  g_a[k] = (self.H[k] - y_train) * self.H[k] * (1 - self.H[k]) 
                
            else:
                g_h[k] = (1/n_samples)*np.matmul(self.weights[k+1].T,g_a[k+1])
                if self.activation == 'sigmoid':
                  g_a[k] = (1/n_samples)*np.multiply(g_h[k],sigmoid_derivative(self.A[k]))
                elif self.activation == 'tanh':
                  g_a[k] = (1/n_samples)*np.multiply(g_h[k],tanh_derivative(self.A[k]))
                elif self.activation == 'relu':
                  g_a[k] = (1/n_samples)*np.multiply(g_h[k],relu_derivative(self.A[k]))
                

            if k == 0:
                g_w[k] = (1/n_samples)*np.matmul(g_a[k],X_train.T) 
            else:
                g_w[k] = (1/n_samples)*np.matmul(g_a[k],self.H[k-1].T)

            g_b[k]  = (1/n_samples)*np.sum(g_a[k], axis=1, keepdims = True)
        return g_w,g_b


    def predict(self, X,y ):
      output =  self.forward_propagation(X)
      out_class=(np.argmax(output,axis=0))
      accuracy = round(accuracy_score(y, out_class) * 100, 3)
      return accuracy
    def fit_predict(self,X_train,y_train,X_val ,y_val ,epochs,learning_rate,optimiser='gd',batch_size = 64,lambd=0.0005): 
      steps = 0
      pre_update_w = np.multiply(self.weights,0)
      pre_update_b = np.multiply(self.biases,0)
      update_w = np.multiply(self.weights,0)
      update_b = np.multiply(self.biases,0)
      vw = 0.0
      vb = 0.0
      eps = 1e-8
      a1 =0.0
      gamma = 0.9
      beta = 0.999
      beta1 = 0.9
      beta2 = 0.999
      m_t, v_t, m_hat_w, v_hat_w, m_b,v_b,m_hat_b,v_hat_b = 0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0 

      for i in range(epochs):
        

            for batch in range(0, X_train.shape[1], batch_size):

              train_b_imag =  X_train[:,batch:batch+batch_size]
              train_l_imag =  y_train_encode[:,batch:batch+batch_size]
              output =  self.forward_propagation(train_b_imag)
              g_weights,g_biases = self.back_propagation(train_b_imag,train_l_imag)
              if optimiser == 'gd':
                 self.weights = self.weights - np.multiply(learning_rate,g_weights) - np.multiply(learning_rate*lambd,self.weights)
                 self.biases = self.biases - np.multiply(learning_rate,g_biases)
             
              if optimiser == 'mgd':
                 
                 update_w = np.multiply(gamma,pre_update_w) + np.multiply(learning_rate,g_weights)
                 self.weights = self.weights - update_w - np.multiply(learning_rate*lambd,self.weights)
                
                 update_b = np.multiply(gamma,pre_update_b) + np.multiply(learning_rate,g_biases)
                 self.biases = self.biases - update_b
                 pre_update_w = update_w
                 pre_update_b = update_b

              if optimiser == 'ngd':
                 
                 cache_w = self.weights
                 cache_b = self.biases
                 self.weights = self.weights - np.multiply(gamma,pre_update_w)
                 self.biases = self.biases - np.multiply(gamma,pre_update_b)
                 output =  self.forward_propagation(train_b_imag)
                 g_weights,g_biases = self.back_propagation(train_b_imag,train_l_imag)

                 update_w = np.multiply(gamma,pre_update_w) + np.multiply(learning_rate,g_weights)
                 cache_w = cache_w - update_w - np.multiply(learning_rate*lambd,cache_w)
                    
                 update_b = np.multiply(gamma,pre_update_b) + np.multiply(learning_rate,g_biases)
                 cache_b = cache_b - update_b
                 
                 self.weights = cache_w
                 self.biases = cache_b
                 pre_update_w = update_w
                 pre_update_b = update_b
                
              if optimiser == 'rmsprop': 

                 vw = np.multiply(vw,beta) + np.multiply(1-beta,np.power(g_weights,2))
                 vb = np.multiply(vb,beta) + np.multiply(1-beta,np.power(g_biases,2))

          
                 update_w = np.multiply(g_weights,learning_rate/np.power(vw+eps,1/2))
                 update_b = np.multiply(g_biases,learning_rate/np.power(vb+eps,1/2))
                 
                 self.weights = self.weights - update_w - np.multiply(learning_rate*lambd,self.weights)
                 self.biases = self.biases - update_b
                    
              if optimiser == 'adam':
                 m_t = np.multiply(beta1,m_t) + np.multiply(1-beta1,g_weights)
                 v_t = np.multiply(beta2,v_t) + np.multiply(1-beta2,np.power(g_weights,2))
                 m_b = np.multiply(beta1,m_b) + np.multiply(1-beta1,g_biases)
                 v_b = np.multiply(beta2,v_b) + np.multiply(1-beta2,np.power(g_biases,2))
                
                 m_hat_w = m_t/(1 - np.power(beta1,i+1))
                 m_hat_b = m_b/(1 - np.power(beta1,i+1))
                
                 v_hat_w = v_t/(1 - np.power(beta2,i+1))
                 v_hat_b = v_b/(1 - np.power(beta2,i+1))
                 update_w = (learning_rate / np.power(v_hat_w + eps, 1/2)) * m_hat_w
                 update_b = (learning_rate / np.power(v_hat_b + eps, 1/2)) * m_hat_b
                 self.weights = self.weights - update_w - np.multiply(learning_rate*lambd,self.weights)
                 self.biases = self.biases - update_b

              if optimiser == 'nadam':
                 
                 self.weights = self.weights - np.multiply(gamma,update_w)
                 self.biases  = self.biases  - np.multiply(gamma,update_b)

                 g_weights,g_biases = self.back_propagation(train_b_imag,train_l_imag)

                 m_t =  np.multiply(beta1,m_t) + np.multiply(1 - beta1,g_weights)
                 v_t =  np.multiply(beta2,v_t) + np.multiply(1 - beta2,np.power(g_weights, 2))

                 m_b =  np.multiply(beta1,m_b) + np.multiply(1 - beta1,g_biases)
                 v_b =  np.multiply(beta2,v_b) + np.multiply(1 - beta2,np.power(g_biases, 2))
                
                 m_hat_w = m_t / (1 - np.power(beta1, i+1)) 
                 v_hat_t = v_t / (1 - np.power(beta2, i+1))

                 m_hat_b = m_b / (1 - np.power(beta1, i+1)) 
                 v_hat_b = v_b / (1 - np.power(beta2, i+1))
  
                 a1 = (1-beta1)/(1-np.power(beta,i+1))
                 update_w = np.multiply(learning_rate/(np.power(v_hat_t + eps,1/2)),(np.multiply(a1,g_weights) + np.multiply(beta1,m_hat_w)))#(a4 + a2))
                 update_b = np.multiply(learning_rate/(np.power(v_hat_b + eps,1/2)),(np.multiply(a1,g_biases)+np.multiply(beta1,m_hat_b) ))#(a5 + a3))
                 self.weights = self.weights - update_w - np.multiply(learning_rate*lambd,self.weights)
                 self.biases = self.biases - update_b
                
            
            #Training loss for full dataset
            predicted_train = self.forward_propagation(X_train)
            predicted_train_label=(np.argmax(predicted_train,axis=0)) 
             
            acc1 = round(accuracy_score(y_train, predicted_train_label) * 100, 3)

            predicted_val = self.forward_propagation(X_val)
            predicted_val_label=(np.argmax(predicted_val,axis=0))
              
            acc2 = round(accuracy_score(y_val, predicted_val_label) * 100, 3)

            a =self.weights[1:len(self.hidden_layer)]
            b = np.sum([(np.sum((a[i]**2).reshape(1,-1))) for i in range(len(a))]) 
            if self.loss_function == 'cross_entropy':
              train_loss = (-np.sum(np.multiply(y_train_encode,np.log(predicted_train)))+((lambd/2.)*b))/y_train.shape[0]
              val_loss = (-np.sum(np.multiply(y_val_encode,np.log(predicted_val)))+((lambd/2.)*b))/y_val.shape[0]
            elif self.loss_function == 'square_loss':
              val_loss =((1/2) * np.sum((y_val_encode- predicted_val)**2))/y_val.shape[0]
              train_loss = (1/2) * np.sum((y_train_encode - predicted_train)**2)/y_train.shape[0]



            print('Epoch {}: training_accuracy = {:.2f} %, training_loss= {:.5f}  ,Validation accuracy = {:.2f} ,Validation loss = {:.5f}'.format(i,acc1,train_loss,acc2,val_loss))
            
            wandb.log({"val_accuracy": acc2,"train_accuracy": acc1,"steps":epochs,"train_loss":train_loss,"val_loss":val_loss},)


         
      return acc1,acc2,train_loss,val_loss

In [8]:
%pip install wandb -q
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [9]:
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize' 
    },
    'parameters': {
        'epochs': {
            'values': [10,15,20]
        },
        'learning_rate': {
            'values': [1e-2,1e-3]
        },
        'optimiser': {
            'values': ['gd','mgd','ngd', 'rmsprop','adam','nadam']
        },
        'activation': {
            'values': ['relu','tanh','sigmoid']
        },
        'hidden_layer': {
            'values': [[32,64],[64,64,64],[32,64,128],[128,128,128],[32,64,128,256]]
        },
        'batch_size':{
            'values':[64,128,256]
        },
        'weight_decay':{
            'values':[0.0,0.5,0.0005]
        },
        'loss_function':{
            'values':['square_loss']
        },
        'initilisation':{
            'values':['xavier']
        }
    }
}

In [10]:
sweep_id = wandb.sweep(sweep_config, entity="moni6264", project="test_mse_50")

Create sweep with ID: 4hhn4fkp
Sweep URL: https://wandb.ai/moni6264/test_mse_50/sweeps/4hhn4fkp


In [11]:
def train():
    #steps = 10
    # Default values for hyper-parameters we're going to sweep over
    config_defaults = {
        'epochs': 5,
        'input_layer': 28*28,
        'output_layer': 10,
        'batch_size':64,
        'weight_decay':0.005,
        'learning_rate': 1e-3,
        'hidden_layer':[16,16,16],
        'learning_rate':1e-2,
        'optimiser':'mgd',
        'activation':'sigmoid',
        'initialisation':'xavier',
        'loss_function':'square_loss'
        
    }

    # Initialize a new wandb run
    wandb.init(project='test_mse_50', entity='moni6264',config=config_defaults)
    
    
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config
    learning_rate = config.learning_rate
    epochs = config.epochs
    hidden_layer = config.hidden_layer
    activation = config.activation
    optimiser = config.optimiser
    input_layer = config.input_layer
    output_layer = config.output_layer
    batch_size = config.batch_size
    weight_decay = config.weight_decay
    loss_function = config.loss_function
    initialisation = config.initilisation

    # Model training here
    sweep_network    = ANN(input_layer, hidden_layer, output_layer,initialisation,activation,loss_function)
    acc1,acc2,train_loss,val_loss  = sweep_network.fit_predict(X_train,y_train,X_val,y_val,epochs,learning_rate,optimiser,batch_size,weight_decay)


In [12]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
wandb.agent(sweep_id, train, count=50)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: grtfqakv with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 256
wandb: 	epochs: 10
wandb: 	hidden_layer: [128, 128, 128]
wandb: 	initilisation: xavier
wandb: 	learning_rate: 0.01
wandb: 	loss_function: square_loss
wandb: 	optimiser: rmsprop
wandb: 	weight_decay: 0.0005
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch 0: training_accuracy = 34.16 %, training_loss= 0.44232  ,Validation accuracy = 33.73 ,Validation loss = 0.44263
Epoch 1: training_accuracy = 35.87 %, training_loss= 0.43438  ,Validation accuracy = 35.17 ,Validation loss = 0.43471
Epoch 2: training_accuracy = 38.34 %, training_loss= 0.42697  ,Validation accuracy = 37.30 ,Validation loss = 0.42740
Epoch 3: training_accuracy = 38.60 %, training_loss= 0.41926  ,Validation accuracy = 37.57 ,Validation loss = 0.41983
Epoch 4: training_accuracy = 39.34 %, training_loss= 0.41092  ,Validation accuracy = 38.18 ,Validation loss = 0.41153
Epoch 5: training_accuracy = 39.69 %, training_loss= 0.40321  ,Validation accuracy = 38.88 ,Validation loss = 0.40383
Epoch 6: training_accuracy = 38.17 %, training_loss= 0.40850  ,Validation accuracy = 36.88 ,Validation loss = 0.40974
Epoch 7: training_accuracy = 42.87 %, training_loss= 0.38805  ,Validation accuracy = 42.37 ,Validation loss = 0.38892
Epoch 8: training_accuracy = 42.37 %, training_loss= 0.3

steps,▁▁▁▁▁▁▁▁▁▁
train_accuracy,▁▂▄▅▅▅▄██▆
train_loss,█▇▆▅▄▃▄▁▃▂
val_accuracy,▁▂▄▄▅▅▄█▇▅
val_loss,█▇▆▅▄▃▄▁▃▂
steps,10
train_accuracy,39.807
train_loss,0.39763
val_accuracy,39.05
val_loss,0.39878


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 9p1jocxc with config:
wandb: 	activation: tanh
wandb: 	batch_size: 64
wandb: 	epochs: 15
wandb: 	hidden_layer: [32, 64, 128, 256]
wandb: 	initilisation: xavier
wandb: 	learning_rate: 0.01
wandb: 	loss_function: square_loss
wandb: 	optimiser: gd
wandb: 	weight_decay: 0.5
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch 0: training_accuracy = 9.98 %, training_loss= 0.45518  ,Validation accuracy = 10.18 ,Validation loss = 0.45440
Epoch 1: training_accuracy = 9.98 %, training_loss= 0.45507  ,Validation accuracy = 10.18 ,Validation loss = 0.45429
Epoch 2: training_accuracy = 9.98 %, training_loss= 0.45487  ,Validation accuracy = 10.18 ,Validation loss = 0.45410
Epoch 3: training_accuracy = 9.98 %, training_loss= 0.45467  ,Validation accuracy = 10.18 ,Validation loss = 0.45392
Epoch 4: training_accuracy = 9.98 %, training_loss= 0.45449  ,Validation accuracy = 10.18 ,Validation loss = 0.45374
Epoch 5: training_accuracy = 9.98 %, training_loss= 0.45431  ,Validation accuracy = 10.18 ,Validation loss = 0.45357
Epoch 6: training_accuracy = 9.98 %, training_loss= 0.45413  ,Validation accuracy = 10.18 ,Validation loss = 0.45341
Epoch 7: training_accuracy = 9.98 %, training_loss= 0.45396  ,Validation accuracy = 10.18 ,Validation loss = 0.45325
Epoch 8: training_accuracy = 9.98 %, training_loss= 0.45380  ,Va

...

Epoch 0: training_accuracy = 10.02 %, training_loss= 0.89976  ,Validation accuracy = 9.78 ,Validation loss = 0.90217
Epoch 1: training_accuracy = 10.01 %, training_loss= 0.89994  ,Validation accuracy = 9.95 ,Validation loss = 0.90050
Epoch 2: training_accuracy = 10.01 %, training_loss= 0.89994  ,Validation accuracy = 9.95 ,Validation loss = 0.90050
Epoch 3: training_accuracy = 9.98 %, training_loss= 0.90015  ,Validation accuracy = 10.13 ,Validation loss = 0.89867
Epoch 4: training_accuracy = 10.09 %, training_loss= 0.89905  ,Validation accuracy = 9.17 ,Validation loss = 0.90831
Epoch 5: training_accuracy = 9.98 %, training_loss= 0.55873  ,Validation accuracy = 10.13 ,Validation loss = 0.55888
Epoch 6: training_accuracy = 10.01 %, training_loss= 0.47564  ,Validation accuracy = 9.95 ,Validation loss = 0.47535
Epoch 7: training_accuracy = 10.09 %, training_loss= 0.46018  ,Validation accuracy = 9.17 ,Validation loss = 0.46076
Epoch 8: training_accuracy = 9.96 %, training_loss= 0.59052  ,Va

steps,▁▁▁▁▁▁▁▁▁▁
train_accuracy,▄▃▃▂█▂▃█▁█
train_loss,█████▃▁▁▃▁
val_accuracy,▅▆▆▇▁▇▆▁█▁
val_loss,█████▃▁▁▃▁
steps,10
train_accuracy,10.093
train_loss,0.45742
val_accuracy,9.167
val_loss,0.458


wandb: Agent Starting Run: jnnh69vy with config:
wandb: 	activation: tanh
wandb: 	batch_size: 64
wandb: 	epochs: 15
wandb: 	hidden_layer: [32, 64]
wandb: 	initilisation: xavier
wandb: 	learning_rate: 0.01
wandb: 	loss_function: square_loss
wandb: 	optimiser: nadam
wandb: 	weight_decay: 0.0005
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch 0: training_accuracy = 10.02 %, training_loss= 0.89976  ,Validation accuracy = 9.78 ,Validation loss = 0.90217
Epoch 1: training_accuracy = 10.02 %, training_loss= 0.89976  ,Validation accuracy = 9.78 ,Validation loss = 0.90217
Epoch 2: training_accuracy = 10.02 %, training_loss= 0.89976  ,Validation accuracy = 9.78 ,Validation loss = 0.90217
Epoch 3: training_accuracy = 10.02 %, training_loss= 0.89976  ,Validation accuracy = 9.78 ,Validation loss = 0.90217
Epoch 4: training_accuracy = 10.02 %, training_loss= 0.89976  ,Validation accuracy = 9.78 ,Validation loss = 0.90217
Epoch 5: training_accuracy = 10.02 %, training_loss= 0.89976  ,Validation accuracy = 9.78 ,Validation loss = 0.90217
Epoch 6: training_accuracy = 10.02 %, training_loss= 0.89976  ,Validation accuracy = 9.78 ,Validation loss = 0.90217
Epoch 7: training_accuracy = 10.02 %, training_loss= 0.89976  ,Validation accuracy = 9.78 ,Validation loss = 0.90217
Epoch 8: training_accuracy = 10.02 %, training_loss= 0.89976  ,V

steps,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
steps,15
train_accuracy,10.024
train_loss,0.89976
val_accuracy,9.783
val_loss,0.90217
